In [1]:
import pandas as pd
import re
import numpy as np


In [2]:
df = pd.read_csv('/Users/Theo/Desktop/sample_table.tsv', sep='\t')

In [3]:
df

,nom,prenom,date_naissance,num_mat,classe,signalement,taille,instruction,instruction_militaire,details,affectation,adresse,dates,etat_civil,decision
0,Hernlen,Louis Fernand,16 novembre 1898,5599,NaN,"['bl.', 'long', 'marr.', 'vert', 'rect.', '78']",178.0,NaN,NaN,Incorporé à compter du 16 avril 1917 arrivé au...,"['115^e Rég^t Infanterie 14757', '156° Rég^t I...","['31-3-21 Pantin 7 rue de la Marine', 'f^le Pa...",[],"['16 novembre 1898', 'Paris', '20^e arr^t', 'l...","['1918', 'Aubervilliers', '5599', '17']"
1,Buisson,Alphonse Emile,21 mars 1887,1031,1903.0,"['ct', 'noirs', 'noirs', 'moyens', '?', 'moyen...",185.0,4,NaN,237du 15e arr^tParti le 1° Octobre 1908 pour l...,"['9e Régt de Cuirassiers', 'Rt de Cuirassiers ...","['Corps. Paris 133 R. du Cherche-Midi 15e', '1...",['Frt le 19/1/22'],"['21 mars 1887', 'Paris', '6^e arr^t', 'seine'...","['08', '1']"
2,Soulat,"Jean, Valentin, Fleuris",31 mars 1890,2878,NaN,"['chatains clair', 'Marrons', 'fuyant', 'Moyen...",NaN,5,NaN,26214 ArrondissementReconnu propre au Service ...,"['Service auxiliaire 395', '20e SEM 1013 90303...","['Ceil Ron Paris 272 Bd Raspail 14e', 'Couge 4...",[],"['31 Mars 1890', 'Augoulême', 'udit', 'e la ch...","['5', '11 (faiblesse)', 'Entre', '12 Bronch dé..."
3,Hubert,Jean Joseph,23 janvier 1873,3820,1893.0,"['et', 'bruns', 'gris', 'ordinaire', 'moyen', ...",167.0,2,NaN,Parti pour la 7^e C^ie du 3^e Escd^on Train le...,"['7^e C^ie du 3e Train', '19 Escadron de Train...","['18 7^bre 1897 Paris 149 Ave Ledru Rolin R', ...",['1Novembre 1er Octobre 1er Octobre 1er Octobr...,"['Paris', '23 janvier 1873', 'e la Seine', 'u ...","['Bon Service Armé', '14']"
4,Legrand,Paul Eugène,5 9^bre 1872,806,NaN,"['et', 'Chatains', 'chatains', 'haut', 'moyen'...",168.0,NaN,NaN,Engagé volontaire pour 4 ans le 12 Mars 1891 a...,"[""16^e Reg^t d'artillerie"", '9^e _ d°', ""Régt...",[],['12 Mars 1904 12 Mars 1910 12 Mar'],"['Paris', '5 9^bre 1872', 'e la Seine', 'u 14^...","['Engagé Volontaire', '4']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Gesmier,"Jean, Baptiste, Auguste, Constant",11 février 1879,3994,NaN,"['et', 'Châtains clair', 'bleus', 'large', 'm...",166.0,3,NaN,Dirigé le 1novembre 1900 sur le. 151^e régime...,"[""151^e régiment d'Infanterie"", 'CM 22^c S^o...","['9 rue de Cheroy 17', '26/2/199 19 Ru de Chè...",['1^er Nov^bre 1^er Nov^bre 1^er Nov^bre 1^er ...,"['Ducey', '11 février 1879', 'la Manche', 'ud...","['Bon pour le service armé', '1ère']"
97,Roy,Henri,15 octobre 1892,3796,NaN,"['chatains', 'marrons', 'moyen', 'gros', 'oval...",155.0,3,NaN,astreint à 2 ans de Service article 15 de la ...,"['67^e REGIMENT D\'INFANTERIE "" 6122']",[],['1^er oct 1915 1^er oct 1927 1^er oct 1939 1^...,"['Paris', '15 octobre 1892', 'e la Seine', 'u ...","['1573', 'u 18^e arrond^t', '11^e', '12. < omi..."
98,Brandy,Léonard Pierre,31 mai 1895,5618,1911.0,"['bruns', 'rond', 'marrons', 'moyen', 'recitil...",158.0,3,NaN,"incorporé le 19 Décembre 1914, service comptan...","['20^e Bataillon de Chasseur à Pied 4453', '12...",['G^w 1. 12 30 Nanterre Seine 51 n° RUe du Lab...,[],"['31 Mai 1895', 'Saint Junien', 'Saint Junien'...","['57', 'Pateaux', '1', '14']"
99,Tauxier,"Octave, Marie Hervé",22 juillet 1873,799,NaN,"['et', 'bruns', 'noirs', 'ordre', 'moyen', 'mo...",165.0,NaN,NaN,Dirigé le 13 9^bre 1891 sur le 101e Rég^t d'...,"[""101^e Rég^t d'Inf^ie"", ""Reg^t d'Inf^ie de D...","[""Etat d'aff^on Paris, 2 rue Pascal D"", ""ie10-...",['1^er 9^bre 1897 1^er 9^bre 1907 1^er 9^bre 1...,"['22 Juillet 1873', 'Charenton du cher', 'u di...","['Bon Dispensé act 21', 'Frère au Service', '2']"


In [4]:
affectation_dico = {"Infanterie": ["infa", "chasseurs a pied", "chasseur a pied",
                                   " ri ", "infanterie", "infie", "infi", "infe", "ifie", "interie" ],
                    "Troupes coloniales" : ["tirail", "goum", "afrique", "spahis", "coloniaux", 
                                            "zouaves", "etranger", "tirailleurs", "colo"],
                    "Aviation" : ["avia", "aero"],
                    "Train" : ["train"],
                    "Genie" : ["genie"],
                    "Cavalerie" : ["dragons", "cuirassiers", "chasseurs", "hussard", "huss", "chasseurs a cheval",
                                   "chas a cheval", "chasseur a cheval", "drag"],
                    "Artillerie" : ["art", "ouvriers dart", "ouv art", "ouv d art", 
                                    "boa", "alc", "r a ", "dartillerie" , "legere portee"],
                    "Corps medical" : ["infirmier"],
                    "Service auxiliaire" : ["aux", "etat major", "coa", "sem", "em" ],
                    "Affectations spéciales" : ["special", "usine"]}

In [5]:
import string
from unidecode import unidecode

# Define a dictionary to map accented characters to their non-accented counterparts
accented_chars = {
    'à': 'a', 'á': 'a', 'â': 'a', 'ã': 'a', 'ä': 'a', 'å': 'a',
    'ç': 'c',
    'è': 'e', 'é': 'e', 'ê': 'e', 'ë': 'e',
    'ì': 'i', 'í': 'i', 'î': 'i', 'ï': 'i',
    'ñ': 'n',
    'ò': 'o', 'ó': 'o', 'ô': 'o', 'õ': 'o', 'ö': 'o',
    'ù': 'u', 'ú': 'u', 'û': 'u', 'ü': 'u',
    'ý': 'y',
}

def remove_accented_characters(text):
    for char, replacement in accented_chars.items():
        text = text.replace(char, replacement)
    return text

def remove_punctuation(text):
    return ''.join(char for char in text if char not in string.punctuation)

def determine_most_frequent_arme(df, keyword_dict):
    def count_keywords(text, keywords):
        count = {key: 0 for key in keywords}
        for keyword in keywords:
            for word in text.split():
                for keyword_value in keyword_dict[keyword]:
                    if keyword_value in word:
                        count[keyword] += 1
        max_count = max(count.values())
        if max_count == 0 or not text.strip():
            return None
        max_keys = [key for key, value in count.items() if value == max_count]
        if len(max_keys) == 1:
            return max_keys[0]
        else:
            last_key = None
            for word in text.split():
                for key in max_keys:
                    if any(keyword_value in word for keyword_value in keyword_dict[key]):
                        last_key = key
            return last_key

    # Unlist the 'affectation' column and join the elements as a single string
    df['affectation'] = df['affectation'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

    # Convert the text to lowercase, remove accented characters, and remove punctuation
    df['affectation'] = df['affectation'].apply(lambda x: remove_punctuation(remove_accented_characters(x.lower())))

    # Define a function to count keywords for each row and return the most frequent one
    df['arme'] = df['affectation'].apply(lambda x: count_keywords(x, keyword_dict))

    return df

In [6]:
df = determine_most_frequent_arme(df, affectation_dico)

In [7]:
df['arme'].value_counts(dropna=False)

Infanterie            57
Artillerie            15
Cavalerie              7
None                   6
Genie                  5
Service auxiliaire     4
Troupes coloniales     4
Train                  2
Corps medical          1
Name: arme, dtype: int64

In [8]:
decision_dico = {"Bon pour le service armé": ["arme", "armé"],
                 "Exempté" : ["dispense", "exempte", "dispensé", "exempté", "dipensé", "dipense"],
                 "Ajourné" : ["ajourne", "ajourné", "sejourne", "sejourné"],
                 "Bon absent" : ["absent"],
                 "Bon pour le serivce auxiliaire" : ["auxil", "auxiliaire", "faible"],
                 "Engagé volontaire" : ["engage", "volontaire", "engagé"]}

In [9]:
decision_dico

{'Bon pour le service armé': ['arme', 'armé'],
 'Exempté': ['dispense',
  'exempte',
  'dispensé',
  'exempté',
  'dipensé',
  'dipense'],
 'Ajourné': ['ajourne', 'ajourné', 'sejourne', 'sejourné'],
 'Bon absent': ['absent'],
 'Bon pour le serivce auxiliaire': ['auxil', 'auxiliaire', 'faible'],
 'Engagé volontaire': ['engage', 'volontaire', 'engagé']}

In [10]:
def determine_extracted_decision(df, decision_dict):
    def count_keywords(text, keywords):
        count = {key: 0 for key in keywords}
        for keyword in keywords:
            for word in text.split():
                for keyword_value in decision_dict[keyword]:
                    if keyword_value in word:
                        count[keyword] += 1
        max_count = max(count.values())
        if max_count == 0 and 'bon' in text.split():
            return 'Bon pour le service armé'
        if max_count == 0 or not text.strip():
            return None
        max_keys = [key for key, value in count.items() if value == max_count]
        if len(max_keys) == 1:
            return max_keys[0]
        else:
            last_key = None
            for word in text.split():
                for key in max_keys:
                    if any(keyword_value in word for keyword_value in decision_dict[key]):
                        last_key = key
            return last_key

    # Unlist the 'decision' column and join the elements as a single string
    df['decision'] = df['decision'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

    # Convert the text to lowercase, remove accented characters, and remove punctuation
    df['decision'] = df['decision'].apply(lambda x: remove_punctuation(remove_accented_characters(x.lower())))

    # Define a function to count keywords for each row and return the most frequent one
    df['extracted_decision'] = df['decision'].apply(lambda x: count_keywords(x, decision_dict))

    return df

In [11]:
df = determine_extracted_decision(df, decision_dico)

In [12]:
df['extracted_decision'].value_counts(dropna=False)

None                              53
Bon pour le service armé          18
Engagé volontaire                 10
Exempté                            8
Bon pour le serivce auxiliaire     6
Bon absent                         4
Ajourné                            2
Name: extracted_decision, dtype: int64

In [13]:
import re

def check_insoumis(df):
    # Use a regular expression to find "insoumis" case-insensitively, allowing it to be stuck to other characters
    df['insoumis'] = df['details'].str.contains(r'\binsoumis\b', case=False, regex=True)

    return df

In [14]:
df = check_insoumis(df)

In [15]:
df["insoumis"].value_counts(dropna = False)

False    97
True      4
Name: insoumis, dtype: int64

In [16]:
import re
import string

def search_certificat_de_bonne_conduite(df):
    df['details_cleaned'] = df['details'].apply(lambda x: remove_punctuation(remove_accented_characters(x.lower())) if x else None)
    
    def find_certificat_conduite(text):
        words_to_find = ["certificat", "bonne", "conduite"]
        found_words = [word for word in words_to_find if re.search(r'[^\w\s]'.format(word), text)]
        if "conduite" in found_words and len(found_words) > 1:
            accorded_words = ["a reçu", "a recu", "accordé", "accorde", "Accorde", "Accordé"]
            refused_words = ["refuse", "refusé", "n'a pas", "n a pas", "Refusé", "Refuse"]
            
            for word in text.split():
                word_stripped = re.sub(r'[^\w\s]', '', word)
                if any(word_stripped.startswith(acc_word) for acc_word in accorded_words):
                    return "Accordé"
                elif word_stripped in refused_words:
                    return "Refusé"
        return None
    
    df['certificat'] = df['details_cleaned'].apply(lambda x: find_certificat_conduite(x) if x else None)
    df.drop(columns=['details_cleaned'], inplace=True)

    return df

In [17]:
df = search_certificat_de_bonne_conduite(df)

In [18]:
df['certificat'].value_counts(dropna=False)

None       66
Accordé    31
Refusé      4
Name: certificat, dtype: int64

In [19]:
df[df['certificat'] == "Accordé"]

,nom,prenom,date_naissance,num_mat,classe,signalement,taille,instruction,instruction_militaire,details,affectation,adresse,dates,etat_civil,decision,arme,extracted_decision,insoumis,certificat
1,Buisson,Alphonse Emile,21 mars 1887,1031,1903.0,"['ct', 'noirs', 'noirs', 'moyens', '?', 'moyen...",185.0,4,NaN,237du 15e arr^tParti le 1° Octobre 1908 pour l...,9e regt de cuirassiers rt de cuirassiers b 7 0...,"['Corps. Paris 133 R. du Cherche-Midi 15e', '1...",['Frt le 19/1/22'],"['21 mars 1887', 'Paris', '6^e arr^t', 'seine'...",08 1,Cavalerie,None,False,Accordé
3,Hubert,Jean Joseph,23 janvier 1873,3820,1893.0,"['et', 'bruns', 'gris', 'ordinaire', 'moyen', ...",167.0,2,NaN,Parti pour la 7^e C^ie du 3^e Escd^on Train le...,7e cie du 3e train 19 escadron de train à par...,"['18 7^bre 1897 Paris 149 Ave Ledru Rolin R', ...",['1Novembre 1er Octobre 1er Octobre 1er Octobr...,"['Paris', '23 janvier 1873', 'e la Seine', 'u ...",bon service arme 14,Train,Bon pour le service armé,False,Accordé
4,Legrand,Paul Eugène,5 9^bre 1872,806,NaN,"['et', 'Chatains', 'chatains', 'haut', 'moyen'...",168.0,NaN,NaN,Engagé volontaire pour 4 ans le 12 Mars 1891 a...,16e regt dartillerie 9e d° régt dartie de co...,[],['12 Mars 1904 12 Mars 1910 12 Mar'],"['Paris', '5 9^bre 1872', 'e la Seine', 'u 14^...",engage volontaire 4,Artillerie,Engagé volontaire,False,Accordé
5,Jondot,Georges,20 août 1889,1610,NaN,"['ct', 'châtains', 'châtains', 'ordinaire', 'c...",165.0,3,exerce,Dirigé le 7octobre 1907 sur le 148^e Rég^t d'i...,148e regt dinfie regt infie chartresparis 460e...,[],['18.1909'],"['20 Août 1889', 'Paris', '5^e Arrond^t', 'Se...",classe dans la 3eme partie de la liste en 1906...,Infanterie,Bon pour le service armé,False,Accordé
7,Mierral,Auguste Claude Antoine,10 octobre 1887,1603,1907.0,"['et', 'noris', 'bruns', 'haut', 'et', 'moyens...",167.0,3,NaN,399 PantinMis en route le 6 Octobre 1908 pour ...,45e régt dinfanterie régt dinfrie a sam 75 1...,"[""le 25-9-10 à Pantin 30 Pl de l'église P^or""...",[],"['10 Octobre 1887', 'Paris ', 'u 19^e Art', 'd...",1 08,Infanterie,None,False,Accordé
11,Loron,Jean François,25 mai 1881,1209,NaN,"['et', 'chât. clair', 'bruns', 'large', 'et', ...",173.0,3,exerce,Dirigé le 15 novembre 1902 sur le 25 Régt d'ar...,25 regt dartillerie 44e artie de corps au mans...,"[""Etat d'aff^on Paris 32 rue du Bièvre ( 5"", '...",['1:9^br 1905 1:9^br 1915 1:9^br 1921 1:9^br 1...,"['25 Mai 1881', 'Chalamont', 'u dit', ""e l'Ain...",bon 1,Artillerie,Bon pour le service armé,False,Accordé
14,Sénéchal,Charles Emile,18 octobre 1888,4014,NaN,"['et', 'bruns', 'noire', 'decouvert', 'et', 'm...",NaN,3,NaN,548du 16 arrondiss^tIncorpo e à cr per du 8 o...,162e reg dinfanterie 2346 162e regt dinfantie ...,"['Cort Paris 127 R la Pompe 16>', '17-3-12 31 ...",['15. 10.1937'],"['Paris', '18 octobre 1888', 'u 16 arrond', 'e...",09 5,Infanterie,None,False,Accordé
17,Normand,Arthur Emille,10 août 1876,319,1896.0,"['et', 'chatains foncé', 'chatains', 'ordinair...",177.0,3,NaN,Dirigé le 15 Novembre 1897 sur le 6e Bataillo...,à pied 7e bataillon dartie a pied 4me bataill...,"['G^ie 1er mars 1901 : Paris', 'G^ie 4 Mars 19...",['1^er Nov^bre 1^er Nov^bre 1^er Nov^bre 1^er ...,"['10 août 1876', 'Campagne-lès-Hesdins', 'Cam...",bon pour le service armé 1ere,Artillerie,Bon pour le service armé,False,Accordé
20,Leclerc,Henri Louis Fédéric,20 octobre 1881,511,1901.0,"['a', 'châtains', 'châtains', 'rond', 'large',...",165.0,NaN,NaN,Partie le 14 Novembre 1902 pour le 153^e Régi...,153e régiment dinfanterie de rern a xye,[],[],"['20 octobre 1881', 'Paris', '7^em arrondt', '...",bon dipensé article 21 aine de veuve 2e,Infanterie,Exempté,False,Accordé
22,Gautier,"René, Ambroise",30 avril 1869,1889,1891.0,"['et', 'châtains', 'châtains', 'bas', 'petit',...",164.0,3,NaN,"Est tenu d'accomplir trois ans de service, son...",72° regt dinfanterie régiment dinfanterie de ...,"['Ge13 fev. 94 Vanves 6 rue de la Gare', '5 X^...",[],"['Paris', '30 Avril 1869', 'e 

In [20]:
grade_dico = {
    1: (["soldat", "sapeur", "hussard", "chasseur", "cycliste", "cavalier", "cuirassier", "cuir", "cannonier"], ["2", "classe"]),
    2: (["soldat", "sapeur", "hussard", "chasseur", "cycliste", "cavalier", "cuirassier", "cuir", "cannonier"], ["1", "classe"]),
    3: (["caporal", "brigadier"], []),
    4: (["chaporal chef", "brigdaier chef", "chaporalchef", "brigdaierchef"], []),
    5: (["sergent", "marechal des logis", "logis"], []),
    6: (["fourier", "sergent chef"], []),
    7: (["adjudant"], []),
    8: (["adjudant chef"], []),
    9: (["aspirant"], []),
    10: (["s lieutenant", "sous lieut", "s lieut", "ss lieu"], []),
    11: (["lieutenant", "lieut"], []),
    12: (["capitaine"], []),
    13: (["commandant"], []),
    14: (["colonel"], [])
}

In [21]:

def search_military_grade(df, grade_dico):
    df['details_cleaned'] = df['details'].apply(lambda x: remove_punctuation(remove_accented_characters(x.lower())) if x else None)

    def find_military_grade(text):
        max_grade = 0
        for grade, (terms, required) in grade_dico.items():
            if grade in (1, 2):
                # For grades 1 and 2, at least one term from the first list and all terms from the second list are required
                pattern = r'(?:\b(?:' + '|'.join(terms) + r')\b.{0,5}(?:' + '|'.join(required) + r'))'
                matches = re.finditer(pattern, text)
                for match in matches:
                    max_grade = max(max_grade, grade)
            else:
                pattern = r'\b(?:' + '|'.join(terms) + r')\b'
                matches = re.finditer(pattern, text)
                for match in matches:
                    max_grade = max(max_grade, grade)

        return max_grade if max_grade > 0 else None

    df['grade'] = df['details_cleaned'].apply(lambda x: find_military_grade(x) if x else None)
    df.drop(columns=['details_cleaned'], inplace=True)

    # Define a dictionary to rename the values in the 'grade' column
    grade_norme_dico = {
        1: "2e classe",
        2: "1er classe",
        3: "caporal",
        4: "caporal-chef",
        5: "sergent",
        6: "sergent-fourier",
        7: "adjudant",
        8: "adjudant-chef",
        9: "aspirant",
        10: "sous-lieutenant",
        11: "lieutenant",
        12: "capitaine",
        13: "commandant",
        14: "colonel"
    }

    df['grade'] = df['grade'].map(grade_norme_dico)

    return df

In [22]:
df = search_military_grade(df, grade_dico)

In [23]:
df['grade'].value_counts(dropna=False)

NaN                48
2e classe          23
caporal             9
sergent             9
lieutenant          5
1er classe          3
sergent-fourier     2
adjudant            1
capitaine           1
Name: grade, dtype: int64

In [24]:
df.columns

Index(['nom', 'prenom', 'date_naissance', 'num_mat', 'classe', 'signalement',
       'taille', 'instruction', 'instruction_militaire', 'details',
       'affectation', 'adresse', 'dates', 'etat_civil', 'decision', 'arme',
       'extracted_decision', 'insoumis', 'certificat', 'grade'],
      dtype='object')

In [25]:
# List of columns to remove
columns_to_remove = ['signalement', 'details', 'affectation', 'adresse', 'dates', 'etat_civil', 'decision']

# Remove the columns
df = df.drop(columns=columns_to_remove)

# Rename 'extracted_decision' to 'decision'
df = df.rename(columns={'extracted_decision': 'decision'})

In [26]:
df.to_excel('/Users/Theo/Desktop/post_traiement.xlsx', index=False)